In [3]:
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import tweepy
import csv
import requests
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns


In [24]:
twitter_arch=pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
twitter_arch.head

<bound method NDFrame.head of                 tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0     892420643555336193                    NaN                  NaN   
1     892177421306343426                    NaN                  NaN   
2     891815181378084864                    NaN                  NaN   
3     891689557279858688                    NaN                  NaN   
4     891327558926688256                    NaN                  NaN   
5     891087950875897856                    NaN                  NaN   
6     890971913173991426                    NaN                  NaN   
7     890729181411237888                    NaN                  NaN   
8     890609185150312448                    NaN                  NaN   
9     890240255349198849                    NaN                  NaN   
10    890006608113172480                    NaN                  NaN   
11    889880896479866881                    NaN                  NaN   
12    889665388333682689          

In [4]:
tweet_list = twitter_arch['tweet_id'].tolist()

In [5]:
#Printing tweet list, commented out, only needed for restart
#print(tweet_list)

In [6]:
list(twitter_arch)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [7]:
#Creating a folder for the image prodiction
folder_name = 'image_pred'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [8]:
#Using get request to store the response into a variable
url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response=requests.get(url)

In [9]:
#Testing response, commented out, only needed for restart
#response.text


In [10]:
#Writing the URL into a folder, creating a tsv folder
with open(os.path.join(folder_name, url.split('/')[-1]), mode= 'wb') as file:
	file.write(response.content)

In [67]:
#Opening the saved csv into a dataframe
image_predict=pd.read_csv('image_pred/image-predictions.tsv', sep='\t')

In [12]:
image_predict.head

<bound method NDFrame.head of                 tweet_id                                            jpg_url  \
0     666020888022790149    https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1     666029285002620928    https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2     666033412701032449    https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3     666044226329800704    https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4     666049248165822465    https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   
5     666050758794694657    https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg   
6     666051853826850816    https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg   
7     666055525042405380    https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg   
8     666057090499244032    https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg   
9     666058600524156928    https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg   
10    666063827256086533    https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg   
11    6660711932215091

In [13]:
list(image_predict)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

In [14]:
#setting up Twitter API


consumer_key = 'xxxx'
consumer_secret = 'xxxxx'
access_token = 'xxxxx'
access_secret = 'xxxxxx'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)


api = tweepy.API(auth, wait_on_rate_limit=True)

In [15]:
#Testing the API
tweet_json = api.get_status(666094000022159362)
print(tweet_json.text)
print(tweet_json.in_reply_to_status_id)
print(tweet_json)

tweet_json = api.get_status(878604707211726852)
print(tweet_json.text)
print(tweet_json.in_reply_to_status_id)
print(tweet_json)


This appears to be a Mongolian Presbyterian mix. Very tired. Tongue slip confirmed. 9/10 would lie down with https://t.co/mnioXo3IfP
None
Status(_api=<tweepy.api.API object at 0x105ed3940>, _json={'created_at': 'Mon Nov 16 03:22:39 +0000 2015', 'id': 666094000022159362, 'id_str': '666094000022159362', 'text': 'This appears to be a Mongolian Presbyterian mix. Very tired. Tongue slip confirmed. 9/10 would lie down with https://t.co/mnioXo3IfP', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 666093996847063040, 'id_str': '666093996847063040', 'indices': [109, 132], 'media_url': 'http://pbs.twimg.com/media/CT5w9gUW4AAsBNN.jpg', 'media_url_https': 'https://pbs.twimg.com/media/CT5w9gUW4AAsBNN.jpg', 'url': 'https://t.co/mnioXo3IfP', 'display_url': 'pic.twitter.com/mnioXo3IfP', 'expanded_url': 'https://twitter.com/dog_rates/status/666094000022159362/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'c

In [16]:
#Requesting id, retweet count and favorite count from the API, writing into a list
#After several unsuccesful tries with id, used id_str, not clear why this works and id does not
tweet_json=[]
errors = []
for tweet in tweet_list:
    try:
       
        tweet_status=api.get_status(tweet)
        tweet_id=tweet_status._json['id_str']
        retweet_count = tweet_status._json['retweet_count']
        favorite_count = tweet_status._json['favorite_count']
      
        tweet_json.append({'tweet_id':int(tweet_id),
                        'retweet_count':int(retweet_count),
                        'favorite_count':int(favorite_count)
        })
                       
       
    except Exception as e:
        print(str(tweet)+str(e))
        errors.append(tweet)

888202515573088257[{'code': 144, 'message': 'No status found with that ID.'}]
873697596434513921[{'code': 144, 'message': 'No status found with that ID.'}]
872668790621863937[{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329[{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744[{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320[{'code': 144, 'message': 'No status found with that ID.'}]
845459076796616705[{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648[{'code': 144, 'message': 'No status found with that ID.'}]
837012587749474308[{'code': 144, 'message': 'No status found with that ID.'}]
827228250799742977[{'code': 144, 'message': 'No status found with that ID.'}]
802247111496568832[{'code': 144, 'message': 'No status found with that ID.'}]
775096608509886464[{'code': 144, 'message': 'No status found with that ID.'}]
770743923962707968[{'code': 144, 'message': 'No status found wit

In [17]:
#Test cell, only needs to run when needed
print(tweet_json)

[{'tweet_id': 892420643555336193, 'retweet_count': 8376, 'favorite_count': 38244}, {'tweet_id': 892177421306343426, 'retweet_count': 6184, 'favorite_count': 32785}, {'tweet_id': 891815181378084864, 'retweet_count': 4092, 'favorite_count': 24691}, {'tweet_id': 891689557279858688, 'retweet_count': 8510, 'favorite_count': 41573}, {'tweet_id': 891327558926688256, 'retweet_count': 9223, 'favorite_count': 39747}, {'tweet_id': 891087950875897856, 'retweet_count': 3067, 'favorite_count': 19951}, {'tweet_id': 890971913173991426, 'retweet_count': 2035, 'favorite_count': 11676}, {'tweet_id': 890729181411237888, 'retweet_count': 18600, 'favorite_count': 64553}, {'tweet_id': 890609185150312448, 'retweet_count': 4209, 'favorite_count': 27424}, {'tweet_id': 890240255349198849, 'retweet_count': 7278, 'favorite_count': 31469}, {'tweet_id': 890006608113172480, 'retweet_count': 7227, 'favorite_count': 30247}, {'tweet_id': 889880896479866881, 'retweet_count': 4902, 'favorite_count': 27408}, {'tweet_id': 8

In [18]:
#Createing df out of tweet_json
tweet_json_df=pd.DataFrame(tweet_json)

In [19]:

tweet_json_df.head()

,favorite_count,retweet_count,tweet_id
0,38244,8376,892420643555336193
1,32785,6184,892177421306343426
2,24691,4092,891815181378084864
3,41573,8510,891689557279858688
4,39747,9223,891327558926688256


In [20]:
#Writing tweet_json into a csv
tweet_json_df.to_csv("tweet_json_us.csv", sep=',')

In [21]:
#Writing tweet_json into a csv file as requested in the description 
#I will use the df and the "tweet_json_my.csv" goint forward
#Turning parameter to writerow into a list
import csv
with open('tweet_json3.csv','w') as f:
    thewriter = csv.writer(f, delimiter=',')
    for item in tweet_json:
        thewriter.writerow([item,])
        

In [22]:
#Opening tweet_json_df from csv so I don't have to rerun Twitter API when server restarts
tweet_json_df_sv=pd.read_csv('tweet_json_us.csv')

In [23]:
tweet_json_df_sv.head()

,Unnamed: 0,favorite_count,retweet_count,tweet_id
0,0,38244,8376,892420643555336193
1,1,32785,6184,892177421306343426
2,2,24691,4092,891815181378084864
3,3,41573,8510,891689557279858688
4,4,39747,9223,891327558926688256


In [24]:

list(tweet_json_df_sv)

['Unnamed: 0', 'favorite_count', 'retweet_count', 'tweet_id']

In [25]:
#Making a copy of tweet_json_df_sv)
tweet_json_df_sv_copy = tweet_json_df_sv.copy()

In [26]:
#Drop the unnamed column
tweet_json_df_sv.drop(columns=['Unnamed: 0'],inplace=True)

In [27]:
#Test dropping of unnamed column
list(tweet_json_df_sv)

['favorite_count', 'retweet_count', 'tweet_id']

In [28]:
tweet_json_df_sv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 3 columns):
favorite_count    2342 non-null int64
retweet_count     2342 non-null int64
tweet_id          2342 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


So far I have three dataframes 
1.)twitter_arch which has ['tweet_id', 'in_reply_to_status_id',
'in_reply_to_user_id','timestamp', 'source','text','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls','rating_numerator' 'rating_denominator' 'name' 'doggo' 'floofer' 'pupper', 'puppo']

2.image_predict which has
['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog']

3.)tweet_json_df_sv which  has
['favorite_count', 'retweet_count', 'tweet_id']



 ## Part 2: Assessing Data for this Project
After gathering each of the above pieces of data, assess them visually and programmatically for quality and tidiness issues. Detect and document at least eight (8) quality issues and two (2) tidiness issues .
Quality issues are issues with content, like inaccurate or duplicate data. Tidiness issues are structural issues, specifically: each variable must be a column, each observation must be a row, and each type of observational unit must be a table
Key points:
You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
Cleaning includes merging individual pieces of data according to the rules of tidy data.
The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.
You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.

## General thoughts before starting analysis:
The analysis center around the dogs.
There is no way to unambiguously identify a dog by name or a dog ID, so  I am assuming there is only one (original) tweet per dog. Owners don’t tweet twice about their dog, so we would use the tweet_id to recognize the dog.
There is no clear guideline what should be done with the data. My assumption: retweets and favorites are an indicator for the popularity of a dog/tweet. Possible learnings from the dataset: Which dog breeds are most popular, which dog types (pupper etc…) are most popular. What is the spread for dogs favorited and retweeted. How often is a dog favorited on average etc...

## Visual assement for twitter_arch dataframe

In [29]:
twitter_arch.head(n=20)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


__Quality issue 1__
Checking dog names first. As mentioned, I don’t consider the actual name to be that important as the dogs will be identified by tweet id.
A first look shows quite a few “None” and also entries that are clearly not names but mistakes (such, a, quite, just etc…).



__Quality issue 2__
The dogtionary mentioned the following type of dogs: doggo, pupper, puppo, floof
The provided file contains the dolumns: doggo, floofer, pupper and puppo.
I am assuming that floof is the correct spelling, the values also state "floof"


__Tidiness issue 1__
In twitter_arch the dog categories (doggo, floofer, pupper and puppo) should be in one column, e.g named "dog category" and have entries for the category.
A lot of dogs have no category. I am assuming this is neither a tidyness nor quality issue but caused by weratedogs not categorizing the dogs(?) 

__Tidiness issue 2__
In twitter_arch, some Twitter URLs are doubled.
I also see non Twitter URLS, specifically for Gofundme. I am assuming, Weratedogs.com users submitted those. I am going to consider those as a quality issue, as we are primarly interested in Twitter. 

### Visual assesment of image predictions

In [30]:
image_predict.head(n=20)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


__Quality issue 3__
Not all images uploaded are for dogs.
This has three different reasons.  Quality issue 3 is about the first two:
1. User upladed drawings or screenshots of dogs
2. Users uploaded animals that are not dogs
https://pbs.twimg.com/media/CT-jNYqW4AAPi2M.jpg

I am making the asumption we are only interested in dogs. If the main interest would be in "whatever user uploads" we would not have a quality issue. As we are interested in dogs and dogs only: Users violated the schema when uploading non dogs. Which makes this a validity issue. While this is a quality issue, the provided data already helps with this case, as it is listed if what is detected on the photo is a dog, probably by checking against a database of dog breeds and types.



__Quality issue 4__
The third reason for non dogs, is that the image recognition failed, a dog is present but the image recognition identifies a more prominent object. Example: A dog was identified as seatbelt
https://pbs.twimg.com/media/CUDeDoWUYAAD-EM.jpg
This is an accuracy problem, a dog is pictured, but not recognized correctly


__Quality issue 5__
Image_pred has a column for img_num. There is only one image URL, it is not clear where the other images are. When I checked a few tweets,  I was not able to see more than one imaget using the Twitter ID either. It looks as we have to continue with one image, so the information in the column is not useful.

### Programmatic assessment

In [31]:
#Looking at source
#On first inspection, source seems to be the same, but the column contains four different URLs
twitter_arch['source'].unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

__Tidiness issue 3__
'Source'should contain "Twitter" and "Vine" the actual URL is had to query. It does not make a difference for us how the tweet was downloaded from Twitter only that it is from Twitter

In [32]:
#How many tweets are in reply
r=twitter_arch['in_reply_to_status_id'].value_counts()
r.sum()

78

__Quality issue 6__ I am making the asumption,  that 'in reply to' means that this tweet is a retweet.
 As it was stated that we are only interested in original tweets. Need to remove retweets

In [33]:
twitter_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [34]:
tweet_json_df_sv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 3 columns):
favorite_count    2342 non-null int64
retweet_count     2342 non-null int64
tweet_id          2342 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


In [35]:
#How many tweets have retweet count of 0?
tweet_json_df_sv.loc[tweet_json_df_sv['retweet_count'] == 0]


,favorite_count,retweet_count,tweet_id
283,149,0,838085839343206401


In [36]:
#Checking the opposite, how mnay tweets have non-zero retweets
tweet_json_df_sv.loc[tweet_json_df_sv['retweet_count'] != 0]

,favorite_count,retweet_count,tweet_id
0,38244,8376,892420643555336193
1,32785,6184,892177421306343426
2,24691,4092,891815181378084864
3,41573,8510,891689557279858688
4,39747,9223,891327558926688256
5,19951,3067,891087950875897856
6,11676,2035,890971913173991426
7,64553,18600,890729181411237888
8,27424,4209,890609185150312448
9,31469,7278,890240255349198849


__Quality issue 7__: According to twitter_arch, there are 181 retweeted tweets, according to tweet_json_df_sv there are 2341 retweeted tweets. I have not yet checked if the tweet id in the two dfs are identical. But even without this, I can make the assumption that the data in twitter_arch is not correct. I trust the data from the Twitter API more. 

__Tidiness issue 4__: twitter_arch and tweet_json should be one dataframe with the following headers: ['tweet_id', 'timestamp', 'source','text’,’expanded_urls','rating_numerator' 'rating_denominator' 'name' 'doggo' 'floofer' 'pupper', 'puppo' favorite count, retweet count]

__Quality issue 8__: Several tweet IDs threw an error when queriying from the API. We can see that from the original 2356 tweet ID in twitter_arch, only 2341 made it into tweet_json_df.
The three above issues can be solved together by creating a new dataframe that only contains tweet ids from both dataframes and does not use the retweeted status from twitter_arch.

## Cleaning data

In [37]:
#Creating a copy from image prediction
image_predict_copy=image_predict.copy()


In [38]:
#Creating a copy for twitter arch
twitter_arch_copy = twitter_arch.copy()

I am not going to fix quality issue 1. There is no meaningful value to replace the incorrect values. The dogs will not be identified by their names, so this seems to be reasonable.

I am not going to fix quality issue 2 separately, as this can be combined with creating a category for dog type

__Test__

In [25]:
#Testing renaming the columns
list(twitter_arch)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [22]:
twitter_arch.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floof,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


__Fixing quality issue 3 and 4 by removing all none dogs out of the dataframe__


__Define__ The image prediction dataframe should only contain dogs and images of dogs. Removing all entries thtat the image prediction does not recognize as dogs

__Code__

In [41]:
image_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [42]:
#Find out in how many cases all three predictions all three true or all three false to see how much ambiguity there is
image_predict.loc[(image_predict['p1_dog'] == image_predict['p2_dog'])&(image_predict['p2_dog'] == image_predict['p3_dog'])]
     

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True
10,666063827256086533,https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg,1,golden_retriever,0.775930,True,Tibetan_mastiff,0.093718,True,Labrador_retriever,0.072427,True
11,666071193221509120,https://pbs.twimg.com/media/CT5cN_3WEAAlOoZ.jpg,1,Gordon_setter,0.503672,True,Yorkshire_terrier,0.174201,True,Pekinese,0.109454,True


1567 out of 2075 are unambiguous.

In [43]:
#Find out in how many cases all three predictions are true
image_predict.loc[(image_predict['p1_dog'] == True) & (image_predict['p2_dog']==True)
                   & (image_predict['p3_dog']==True)]



,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True
10,666063827256086533,https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg,1,golden_retriever,0.775930,True,Tibetan_mastiff,0.093718,True,Labrador_retriever,0.072427,True
11,666071193221509120,https://pbs.twimg.com/media/CT5cN_3WEAAlOoZ.jpg,1,Gordon_setter,0.503672,True,Yorkshire_terrier,0.174201,True,Pekinese,0.109454,True
12,666073100786774016,https://pbs.twimg.com/media/CT5d9DZXAAALcwe.jpg,1,Walker_hound,0.260857,True,English_foxhound,0.175382,True,Ibizan_hound,0.097471,True


In [44]:
#How often are all three false?
image_predict.loc[(image_predict['p1_dog'] == False) & (image_predict['p2_dog']==False)
                   & (image_predict['p3_dog']==False)]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,4.588540e-02,False,terrapin,1.788530e-02,False
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,3.391940e-02,False,partridge,5.206580e-05,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False,desk,8.554740e-02,False,bookcase,7.947970e-02,False
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False,otter,1.525000e-02,False,great_grey_owl,1.320720e-02,False
25,666362758909284353,https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg,1,guinea_pig,0.996496,False,skunk,2.402450e-03,False,hamster,4.608630e-04,False
29,666411507551481857,https://pbs.twimg.com/media/CT-RugiWIAELEaq.jpg,1,coho,0.404640,False,barracouta,2.714850e-01,False,gar,1.899450e-01,False
45,666786068205871104,https://pbs.twimg.com/media/CUDmZIkWcAAIPPe.jpg,1,snail,0.999888,False,slug,5.514170e-05,False,acorn,2.625800e-05,False
50,666837028449972224,https://pbs.twimg.com/media/CUEUva1WsAA2jPb.jpg,1,triceratops,0.442113,False,armadillo,1.140710e-01,False,common_iguana,4.325530e-02,False
51,666983947667116034,https://pbs.twimg.com/media/CUGaXDhW4AY9JUH.jpg,1,swab,0.589446,False,chain_saw,1.901420e-01,False,wig,3.450970e-02,False
53,667012601033924608,https://pbs.twimg.com/media/CUG0bC0U8AAw2su.jpg,1,hyena,0.987230,False,African_hunting_dog,1.260080e-02,False,coyote,5.735010e-05,False


In [45]:
image_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [46]:
#For how many dogs is the first image recognized as a dog?
image_predict.loc[(image_predict['p1_dog'] == True)]
   

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True
10,666063827256086533,https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg,1,golden_retriever,0.775930,True,Tibetan_mastiff,0.093718,True,Labrador_retriever,0.072427,True
11,666071193221509120,https://pbs.twimg.com/media/CT5cN_3WEAAlOoZ.jpg,1,Gordon_setter,0.503672,True,Yorkshire_terrier,0.174201,True,Pekinese,0.109454,True


In [47]:
#How many cases one of the prediction is a dog?
image_predict.loc[(image_predict['p1_dog'] == True) | (image_predict['p2_dog']==True)
                   | (image_predict['p3_dog']==True)]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True
10,666063827256086533,https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg,1,golden_retriever,0.775930,True,Tibetan_mastiff,0.093718,True,Labrador_retriever,0.072427,True


__Dicussing the approach__ There would be several ways of solving this. 
-  Select all dogs where all three predictions are True
-  Select dogs where the first prediction is true
-  Select dogs where at least one predictio is true.

I am going with the most strict approach, all three predictions have to be true.
We are going to loose values, but the chance to have correct dog predictions increases.

In [68]:
#Creating a new dataframe that only contains the rows for dogs where all three predictions see a dog
image_predict_clean=image_predict[(image_predict['p1_dog'] == True) & (image_predict['p2_dog']==True)
                   & (image_predict['p3_dog']==True)]


__Testing__

In [69]:
#Testing. How mnay cases all three of the prediciton are true? Should be all
image_predict_clean.loc[(image_predict_clean['p1_dog'] == True) & (image_predict_clean['p2_dog']==True)
                   & (image_predict_clean['p3_dog']==True)]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True
10,666063827256086533,https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg,1,golden_retriever,0.775930,True,Tibetan_mastiff,0.093718,True,Labrador_retriever,0.072427,True
11,666071193221509120,https://pbs.twimg.com/media/CT5cN_3WEAAlOoZ.jpg,1,Gordon_setter,0.503672,True,Yorkshire_terrier,0.174201,True,Pekinese,0.109454,True
12,666073100786774016,https://pbs.twimg.com/media/CT5d9DZXAAALcwe.jpg,1,Walker_hound,0.260857,True,English_foxhound,0.175382,True,Ibizan_hound,0.097471,True


In [ ]:

image_predict_clean.info()

image_predict_clean has 1243 rows, for 1234 the image prediction is for a dog.

In [51]:
twitter_arch.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floof,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


__Solving Tidyness issue 1 and quality issue 2__
Summarizing dog categories in twitter_arch. Incorrect cateogry 'floofer" will be removed through this.


__Define__ Add a new category dog_type, remove the three separate categories for dogs

__Code__

In the cells below, Checking how often a dog is two types

In [26]:
#Copying to havea df for cleaning
twitter_arch_c=twitter_arch

In [27]:
twitter_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [28]:
twitter_arch_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [37]:
twitter_arch_c.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [29]:
twitter_arch_c.loc[(twitter_arch_c['pupper'] == 'pupper') & (twitter_arch_c['floofer']=='floof')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [30]:
twitter_arch_c.loc[(twitter_arch_c['puppo'] == 'puppo') & (twitter_arch_c['floofer']=='floof')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [31]:
twitter_arch_c.loc[(twitter_arch_c['doggo'] == 'doggo') & (twitter_arch_c['floofer']=='floof')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [32]:
twitter_arch_c.loc[(twitter_arch_c['doggo'] == 'doggo') & (twitter_arch_c['floofer']=='floof')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [33]:
twitter_arch_c.loc[(twitter_arch_c['pupper'] == 'pupper') & (twitter_arch_c['puppo']=='puppo')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [34]:
twitter_arch_c.loc[(twitter_arch_c['pupper'] == 'pupper') & (twitter_arch_c['doggo']=='doggo')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dido. She's playing the lead role in ""...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686...,13,10,Dido,doggo,None,pupper,None
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have Burke (pupper) and Dexter (doggo)...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460...,12,10,None,doggo,None,pupper,None
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like doggo, like pupper version 2. Both 11/10 ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048...,11,10,None,doggo,None,pupper,None
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,doggo,None,pupper,None
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pinot. He's a sophisticated doggo. You...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/785639753...,10,10,Pinot,doggo,None,pupper,None
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,doggo,None,pupper,None
778,775898661951791106,NaN,NaN,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: Like father (doggo), like son (...",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,https://twitter.com/dog_rates/status/733109485...,12,10,None,doggo,None,pupper,None
822,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is just downright precious...,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,https://twitter.com/dog_rates/status/741067306...,12,10,just,doggo,None,pupper,None
889,759793422261743616,NaN,NaN,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Meet Maggie &amp; Lila. Maggie is the doggo, L...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/759793422...,12,10,Maggie,doggo,None,pupper,None
956,751583847268179968,NaN,NaN,2016-07-09 01:08:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please stop sending it pictures that don't eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/751583847...,5,10,None,doggo,None,pupper,None


Some dogs are both doggo and pupper, let's see how many doggos and puppers there are

In [35]:
twitter_arch['pupper'].value_counts()


None      2099
pupper     257
Name: pupper, dtype: int64

In [36]:
twitter_arch['doggo'].value_counts()

None     2259
doggo      97
Name: doggo, dtype: int64

I am going to disregard the double doggo and puppers, assuming this is a mistake.
The double categoriezed dogs will end up being 'doggos'

In [38]:
#Creating a function to check for dog type
def build_category(a):
    if a['doggo'] == 'doggo':
         return 'doggo'
    elif a['floofer'] == 'floof':
         return 'floof'
    elif a['pupper'] == 'pupper':
        return 'pupper'
    elif a['puppo'] == 'puppo':
        return 'puppo'
    else:
        return 'None'

In [39]:
#Applying the function to the new dataframe and creating a column for dog type
twitter_arch_c['dog_type']=twitter_arch_c.apply(build_category, axis=1)

In [40]:
#Checking result
twitter_arch_c.head(n=100)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,dog_type
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None,doggo


__Testing__

In [41]:
#Testing, are there dogs in 'doggo' that have not the dop_type doggo
twitter_arch_c.loc[(twitter_arch_c['doggo'] == 'doggo') & (twitter_arch_c['dog_type']!='doggo')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,dog_type


In [43]:
twitter_arch_c.loc[(twitter_arch_c['floofer'] == 'floof') & (twitter_arch_c['dog_type']!='floof')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,dog_type


In [44]:
twitter_arch_c.loc[(twitter_arch_c['pupper'] == 'pupper') & (twitter_arch_c['dog_type']!='pupper')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,dog_type
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dido. She's playing the lead role in ""...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686...,13,10,Dido,doggo,None,pupper,None,doggo
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have Burke (pupper) and Dexter (doggo)...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460...,12,10,None,doggo,None,pupper,None,doggo
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like doggo, like pupper version 2. Both 11/10 ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048...,11,10,None,doggo,None,pupper,None,doggo
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,doggo,None,pupper,None,doggo
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pinot. He's a sophisticated doggo. You...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/785639753...,10,10,Pinot,doggo,None,pupper,None,doggo
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,doggo,None,pupper,None,doggo
778,775898661951791106,NaN,NaN,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: Like father (doggo), like son (...",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,https://twitter.com/dog_rates/status/733109485...,12,10,None,doggo,None,pupper,None,doggo
822,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is just downright precious...,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,https://twitter.com/dog_rates/status/741067306...,12,10,just,doggo,None,pupper,None,doggo
889,759793422261743616,NaN,NaN,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Meet Maggie &amp; Lila. Maggie is the doggo, L...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/759793422...,12,10,Maggie,doggo,None,pupper,None,doggo
956,751583847268179968,NaN,NaN,2016-07-09 01:08:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please stop sending it pictures that don't eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/751583847...,5,10,None,doggo,None,pupper,None,doggo


In [45]:
twitter_arch_c.loc[(twitter_arch_c['puppo'] == 'puppo') & (twitter_arch_c['dog_type']!='puppo')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,dog_type
191,855851453814013952,NaN,NaN,2017-04-22 18:31:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a puppo participating in the #ScienceMa...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/855851453...,13,10,None,doggo,None,None,puppo,doggo


All dogs had a category assigned, the double categorization did create a few error in this test, but this is expected.

__Define__ Remove the dog columns

__Code__

In [46]:
#Removing doggo, floofer, pupper and puppo column
twitter_arch_c=twitter_arch_c.drop(columns=['doggo', 'floofer','pupper','puppo'])

__Test__

In [49]:
#Testing removal
list(twitter_arch_c)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'dog_type']

__Tidyness issue 2__ In twitter_arch, some Twitter URLs are doubled

__Define__ Remove doubled URLs in twitter archive

__Code__

In [50]:
#Running test for two entries that have double URLs. Will rerun test after updating the dataframe
twitter_arch_c.loc[4,'expanded_urls']


'https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1'

In [51]:
twitter_arch_c.loc[6,'expanded_urls']

'https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1'

If the value is a double URL, we want to remove the first part (before the ',') to make sure we have the Twitter URL and not the gofundme URL.

In [52]:
twitter_arch_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
dog_type                      2356 non-null object
dtypes: float64(4), int64(3), object(7)
memory usage: 257.8+ KB


In [53]:
# As I am running a string manipulation to strip out the first URL, need to convert expanded URLs to string first
twitter_arch_c['expanded_urls']=twitter_arch_c['expanded_urls'].astype(str)

In [54]:
#Using python split, to split the string and return the last element
twitter_arch_c['expanded_urls'] = twitter_arch_c['expanded_urls'].map(lambda x: x.split(',',1)[-1])

__Test__

In [55]:
#Repeating test
twitter_arch_c.loc[4,'expanded_urls']

'https://twitter.com/dog_rates/status/891327558926688256/photo/1'

In [56]:
twitter_arch_c.loc[6,'expanded_urls']

'https://twitter.com/dog_rates/status/890971913173991426/photo/1'

__Tidiness issue 3__ 'Source' should contain  only "Twitter" or "Vine"

__Define__ Changing source to either "Twitter" or "Vine" 

__Code__

In [57]:
twitter_arch_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2356 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
dog_type                      2356 non-null object
dtypes: float64(4), int64(3), object(7)
memory usage: 257.8+ KB


In [58]:
#Creating a function to check for twitter and vine
def check_source(a):
    if 'twitter' in a['source']:
         return 'Twitter'
    elif 'vine' in a['source']:
         return 'Vine'
    else:
        return 'Undefined'

In [59]:
#Applying the function
twitter_arch_c['source_type']=twitter_arch_c.apply(check_source, axis=1)

In [60]:
twitter_arch_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 15 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2356 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
dog_type                      2356 non-null object
source_type                   2356 non-null object
dtypes: float64(4), int64(3), object(8)
memory usage: 276.2+ KB


__Testing__

In [61]:
#Testing the result
twitter_arch_c.loc[twitter_arch_c['source_type'] == 'Vine']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_type,source_type
529,808344865868283904,NaN,NaN,2016-12-12 16:16:49 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Seamus. He's very bad at entering pool...,NaN,NaN,NaN,https://vine.co/v/5QWd3LZqXxd,11,10,Seamus,None,Vine
562,802600418706604034,NaN,NaN,2016-11-26 19:50:26 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Bailey. She has mastered the head tilt...,NaN,NaN,NaN,https://vine.co/v/5FwUWjYaW0Y,11,10,Bailey,None,Vine
657,791774931465953280,NaN,NaN,2016-10-27 22:53:48 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Vine will be deeply missed. This was by far my...,NaN,NaN,NaN,https://vine.co/v/ea0OwvPTx9l,14,10,None,None,Vine
672,789903600034189313,NaN,NaN,2016-10-22 18:57:48 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Ralphy. His dreams were just shattered...,NaN,NaN,NaN,https://vine.co/v/5wPT1aBxPQZ,13,10,Ralphy,pupper,Vine
699,786286427768250368,NaN,NaN,2016-10-12 19:24:27 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Arnie. He's afraid of his own bark. 12...,NaN,NaN,NaN,https://vine.co/v/5XH0WqHwiFp,12,10,Arnie,None,Vine
713,784183165795655680,NaN,NaN,2016-10-07 00:06:50 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Reginald. He's one magical puppo. Aero...,NaN,NaN,NaN,https://vine.co/v/5ghHLBMMdlV,12,10,Reginald,puppo,Vine
714,784057939640352768,NaN,NaN,2016-10-06 15:49:14 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Balto. He's very content. Legendary to...,NaN,NaN,NaN,https://vine.co/v/5gKxeUpuKEr,12,10,Balto,None,Vine
731,781655249211752448,NaN,NaN,2016-09-30 00:41:48 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Combo. The daily struggles of being a ...,NaN,NaN,NaN,https://vine.co/v/5rt6T3qm7hL,11,10,Combo,doggo,Vine
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,doggo,Vine
746,780074436359819264,NaN,NaN,2016-09-25 16:00:13 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Here's a doggo questioning his entire existenc...,NaN,NaN,NaN,https://vine.co/v/5nzYBpl0TY2,10,10,None,doggo,Vine


__Define__ Remove the column for source

__Code__

In [62]:
#Removing source
twitter_arch_c=twitter_arch_c.drop(columns=['source'])

__Test__

In [63]:
#Checking for removal of column
twitter_arch_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2356 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
dog_type                      2356 non-null object
source_type                   2356 non-null object
dtypes: float64(4), int64(3), object(7)
memory usage: 257.8+ KB


In [65]:
image_predict_clean.head()


NameError: name 'image_predict_clean' is not defined

__Solving for Quality issue 5__
Image_predict has a column for img_num.


__Define__ Remove the img_num column, information is not valuable for the analysis

__Code__

In [ ]:
image_predict_clean=image_predict_clean.drop(columns=['img_num'])

__Test__

In [ ]:
#Checking for removed column
image_predict_clean.info()

__Solving for Quality issue 6__ Removing retweets from twitter_arch_c


__Define__ Remove retweets from twitter archive as we do want to exclude those from the analysis

__Code__

In [ ]:
twitter_arch_c.info()

In [ ]:
#Applying to twitter_arc_c
twitter_arch_c = twitter_arch_c[twitter_arch_c['retweeted_status_id'].isnull()]

__Test__

In [ ]:
#Testing the removal
twitter_arch_c.info()
#2356-181=2175. 181 rows  were not null were successfully dropped

In [ ]:
twitter_arch_c.info()

In [ ]:
#Testing for rows that have a null value for retweeted status
twitter_arch_c.loc[twitter_arch_c['retweeted_status_id'].notnull()]

__Define__ Dropping columns for retweeted_status_id , retweeted_status_user_id and retweeted_status_timestamp

__Code__

In [ ]:
#Dropping retweeted_status_id , retweeted_status_user_id and retweeted_status_timestamp
twitter_arch_c=twitter_arch_c.drop(columns=['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp' ])

__Test__

In [ ]:
twitter_arch_c.info()

__Quality issue 7 & 8, Tidynes issue 4__

Solving for Quality issue 7 (incorrect retweet count in twitter_arch), quality issue 8 (tweet ids not returned from API) and tidyness issue 4 (twitter_arch and tweet_json_df should be one dataframe)

__Define__ Combine the API respsonse and the twitter archive

__Code__

Check which tweet ids are in twitter_arch_c and in tweet_json_df_sv

In [ ]:
twitter_arch_c.info()

In [ ]:
tweet_json_df_sv.info()

In [ ]:
#Are there duplicated tweet ids in twitter arch?
c_duplicate=twitter_arch_c['tweet_id'].duplicated()
c_duplicate.value_counts()

In [ ]:
#Duplicated tweet ids in tweet_json?
d_duplicates=tweet_json_df_sv['tweet_id'].duplicated()
d_duplicates.value_counts()

twitter_arch_c and tweet_json_df_sv do not contain dupliate tweet ids

In [ ]:
twitter_arch_combined=pd.merge(twitter_arch_c,tweet_json_df_sv, on='tweet_id')

__Test__

In [ ]:
#Visual check of the new dataframe
twitter_arch_combined.head(n=50)

In [ ]:
twitter_arch_combined.info()

__Define__ Drop columns in_reply_to_status_id and in_reply_to_user_id

In [ ]:
#Dropping columns in_reply_to_status_id, in_reply_to_user_id 
twitter_arch_combined=twitter_arch_combined.drop(columns=['in_reply_to_status_id','in_reply_to_user_id'])


In [ ]:
twitter_arch_combined.info()

In [ ]:
#Dropping Unnamed: 0
twitter_arch_combined=twitter_arch_combined.drop(columns=['Unnamed: 0'])

__Test__

In [ ]:
#Checking for columns in the new dataframe
twitter_arch_combined.info()

In [ ]:
#Writing twitter_arch_combined into a csv
twitter_arch_combined.to_csv("twitter_archive_master.csv", sep=',')


In [ ]:
image_predict_clean.info()

__Define__ Should the tweet ids which are in image prediciton dataframe but not in the combined dataframe be removed?

__Code__

In [ ]:
#Are the tweet IDs in image_predict_clean all in twitter_arch_combined?
values=image_predict_clean['tweet_id'].isin(twitter_arch_combined['tweet_id'])
values.value_counts()


In [ ]:
#Image_predict already has less entries as I removed the  ambiguous dog predictions. 
#Is it worth removing the tweet IDs not in the Twitter_arch? 
#My decision is to remove those, as I consider the twitter_arch to be the master file.

image_predict_clean2=image_predict_clean[image_predict_clean['tweet_id'].isin(twitter_arch_combined['tweet_id'])==True]

__Test__

In [ ]:
#Testing. New dataframe shoudl contain 1202 rows
image_predict_clean2.info()

In [ ]:
image_predict_clean2.head()

In [ ]:
#Writing image_predict_clean2 into a csv
image_predict_clean2.to_csv("twitter_archive_image_pred.csv", sep=',')

### Storing, Analyzing, and Visualizing Data for this Project
Analyze and visualize your wrangled data in your wrangle_act.ipynb Jupyter Notebook. At least three (3) insights and one (1) visualization must be produced.

__Insight and visualization 1__: How is retweet_count and favorite count spread out

In [ ]:
twitter_arch_combined['retweet_count'].mean()

In [ ]:
twitter_arch_combined['retweet_count'].quantile([.25,.5,.75])


In [ ]:
twitter_arch_combined['favorite_count'].quantile([.25,.5,.75])

In [ ]:
#Finding highest value for retweet  count
twitter_arch_combined['retweet_count'].max()

In [ ]:
#Finding highest value for favorite count
twitter_arch_combined['favorite_count'].max()

In [ ]:
#y axis was restricted to 40000 to remove outliers

twitter_arch_combined.boxplot(column=['retweet_count', 'favorite_count'])
plt.ylim(0,40000)


__Findings__ Retweet count and Favorite count are spread very unevenly, the boxplots show a long tail of outliers. 

__Insight and visualization 2__: How are ratings (from weratedogs) and tweet favorites related?


In [ ]:
twitter_arch_combined.info()

In [ ]:
twitter_arch_combined.info()

In [ ]:
#What is the max rating numerator
twitter_arch_combined['rating_numerator'].max()

In [ ]:
#This is another quality issue. At this point I am not going to clean it up
#But disregard unrealistic ratings for my analysis

In [ ]:
#changin rating numerator to int
pd.to_numeric(twitter_arch_combined['rating_numerator'], errors='coerce')

In [ ]:
#From checking the ratings, I am assiuming 13 is the max, testing
r=twitter_arch_combined['rating_numerator']>14
r.value_counts()

In [ ]:
#Only 26 dogs have a higher rating than 13, I am making the asumption that ratings over 13 are not valid
#What is a realstic minimum?
r=twitter_arch_combined['rating_numerator']<3
r.value_counts()

In [ ]:
#Only 19 dogs have a rating lower than 4. While it seems that it is possible to rate lower than 4
#I am going to focus on ratings between 4 and 13, to exclude outliers.


In [ ]:
#Creating a dataframe from 4-13
twitter_arch_4_13_r=twitter_arch_combined[(twitter_arch_combined['rating_numerator']<14) & (twitter_arch_combined['rating_numerator']>3)]



In [ ]:
twitter_arch_4_13_r.info()

In [ ]:
#How many dogs fall into the numerator categories?


twitter_arch_4_13_r.groupby('rating_numerator')['tweet_id'].count()

In [ ]:
#What is the median and the .25 and .75 quantile?
twitter_arch_4_13_r['rating_numerator'].quantile([.25,.5,.75])

In [ ]:
#Mean by rating numerator using round for rounding

round(twitter_arch_4_13_r.groupby('rating_numerator')['favorite_count'].mean())

In [ ]:
#Visualization of mean by rating
data=round(twitter_arch_4_13_r.groupby('rating_numerator')['favorite_count'].mean())
plt.plot(data, 'k--')
plt.xlabel('Rating Numerator')
plt.ylabel('Mean Favorite Count')



__Findings__
Ratings are very generous with a majority getting high ratings, the median being a rating or 11
The ratings on Weratedogs do correspond with high favorites on twitter as the line graph shows.


In [ ]:
#df_under40.groupby('age_bracket').count()['no_Show_int']

__Insights and Visualization 3: Most popular dog breeds__
Using the image predictions, which dogs get most favorited and retweeted?

In [ ]:
#I am going to create a df  with only tweet id, favorite count 

#and retweet count to merge with the image prediction

In [ ]:
image_predict_clean2.info()

In [ ]:
#image_predict_clean2=image_predict_clean[image_predict_clean['tweet_id'].isin(twitter_arch_combined['tweet_id'])==True]
twitter_arch_combined2=twitter_arch_combined.drop(columns=['text','expanded_urls','rating_numerator','rating_denominator','source_type'])


In [ ]:
#Also drop timestamp
twitter_arch_combined2=twitter_arch_combined2.drop(columns=['timestamp'])


In [ ]:
twitter_arch_combined2=twitter_arch_combined2.drop(columns=['name','dog_type'])

In [ ]:
twitter_arch_combined2.info()

In [ ]:
image_predict_clean.info()

In [ ]:
#Creating the new dataframe using join
image_predict_fav_retweet=twitter_arch_combined2.set_index('tweet_id').join(image_predict_clean.set_index('tweet_id'),how='inner')


In [ ]:
image_predict_fav_retweet.info()

In [ ]:
image_predict_fav_retweet.groupby('p1')['retweet_count'].sum().sort_values().tail(10)


In [ ]:
retweet_dogs=image_predict_fav_retweet.groupby('p1')['retweet_count'].sum().sort_values().tail(10)
retweet_dogs.plot.bar()
plt.xlabel('Dog breed')
plt.ylabel('Retweet count')
plt.title('10 most popular dog breeds by Retweet count')


In [ ]:
fav_dogs=image_predict_fav_retweet.groupby('p1')['favorite_count'].sum().sort_values().tail(10)
fav_dogs.plot.bar()
plt.xlabel('Dog breed')
plt.ylabel('Favorite count')
plt.title('10 most popular dog breeds by Favorite count')



__Findings__
Golden retrievers are in the lead, for favorite and retweet count.